In [101]:
## Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import datetime, calendar
from datetime import timedelta
import matplotlib.patches as mpatches
%matplotlib tk

data = pd.read_pickle('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/entwuerfe/xls_testruns/doe_pickle_1458.pkl') # this one for average times

In [102]:
print(data.columns)

Index(['tm', 'dt', 'yy', 'mm', 'ww', 'wd', 'dd', 'xl', 'hh', 'an', 'vb', 'vl',
       'ht', 'tt', 'acw', 'bz', 'dectt', 'decht', 'decacw'],
      dtype='object')


In [103]:
# functions that will be apllied on grouping, left out dt bz and hh; dt and bz should be indices, hh can be dropped
colfunx_collapse_dt={'yy':'first', 'mm':'first', 'ww':'first', 'wd':'first', 'dd':'first', 'xl':'first', 'an':'sum', 'vb':'sum', 'vl':'sum', 'ht':'sum', 'tt':'sum', 'acw':'sum','decht':'sum', 'dectt':'sum', 'decacw':'sum'}
# group data by day and bearbzeit
data_grouped=data.groupby(['dt','bz']).agg(colfunx_collapse_dt)

In [104]:
# dates without neben- or kernzeit should contain each index, but fill them with null. Solution see https://stackoverflow.com/questions/17287933/filling-in-date-gaps-in-multiindex-pandas-dataframe
dg=data_grouped.unstack('dt').fillna(0).stack('dt')
dg2=dg.reorder_levels(['dt','bz'])

In [105]:
# averages as their own columns
dg2['aht']=(dg2['ht']/dg2['vb']).fillna(0)
dg2['att']=(dg2['tt']/dg2['vb']).fillna(0)
dg2['aacw']=(dg2['acw']/dg2['vb']).fillna(0)

# function for conversion of excel-floats to timedelta objects
def float_to_delta(xlfloat):
    deltatime=timedelta(xlfloat)
    return deltatime
dg2[['td_att','td_aht','td_aacw']] = dg2[['att', 'aht', 'aacw']].applymap(float_to_delta)

# reorder columns for better comprehensibility, left out the decimal times and the excel-averages
col_order=['xl','yy','mm','dd','ww','wd','an','vb','vl','ht','tt','acw','td_aht','td_att','td_aacw']
dg2=dg2[col_order]



In [106]:
kernzeit=dg2.xs('k', level=1, drop_level=False)
nebnzeit=dg2.xs('n', level=1, drop_level=False)
# both frames share the same index (level 0, i.e. datetime.date)

In [107]:
kernzeit.tail()
#kernzeit.replace(0, np.nan)

,,xl,yy,mm,dd,ww,wd,an,vb,vl,ht,tt,acw,td_aht,td_att,td_aacw
dt,bz,,,,,,,,,,,,,,,
2017-10-04,k,43012.0,2017.0,10.0,4.0,40.0,Wed,326.0,268.0,58.0,0.726447,0.600822,0.125625,00:03:54.197761,00:03:13.697761,00:00:40.500000
2017-10-05,k,43013.0,2017.0,10.0,5.0,40.0,Thu,303.0,257.0,46.0,0.739329,0.627847,0.111481,00:04:08.552529,00:03:31.073930,00:00:37.478599
2017-10-06,k,43014.0,2017.0,10.0,6.0,40.0,Fri,371.0,332.0,39.0,0.971354,0.815104,0.156250,00:04:12.786145,00:03:32.123494,00:00:40.662651
2017-10-07,k,43015.0,2017.0,10.0,7.0,40.0,Sat,66.0,50.0,16.0,0.167535,0.140706,0.026829,00:04:49.500000,00:04:03.140000,00:00:46.360000
2017-10-08,k,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.000000,0.000000,00:00:00,00:00:00,00:00:00


In [108]:
XAchse=kernzeit.index.levels[0]
YAchse=kernzeit['td_aht']

In [109]:
ax=plt.subplot(111)
ax.plot(kernzeit.index.levels[0],,ls='-',color='red')

SyntaxError: invalid syntax (<ipython-input-109-d2bd2776d192>, line 2)

In [114]:
colfunx_collapse_dt2={'yy':'first', 'mm':'first', 'wd':'first', 'dd':'first', 'xl':'first', 'an':'sum', 'vb':'sum', 'vl':'sum', 'ht':'sum', 'tt':'sum', 'acw':'sum','decht':'sum', 'dectt':'sum', 'decacw':'sum'}
data_grouped_byweek=data_grouped=data.groupby(['dt','bz','ww']).agg(colfunx_collapse_dt2)

In [112]:
#data_grouped_byweek

In [129]:
def get_year(dtobject):
    jahr=dtobject.year
    return jahr

colfunx_collapse_dt2={'date_dt':'get_year','yy':'first', 'mm':'first', 'an':'sum', 'vb':'sum', 'vl':'sum', 'ht':'sum', 'tt':'sum', 'acw':'sum'}
col_order2=['yy','mm','an','vb','vl','ht','tt','acw']
data['date_dt']=data.index
dg3=data.groupby(['ww','bz']).agg(colfunx_collapse_dt2)
dg4=dg3.unstack('bz').fillna(0).stack('bz')
dg5=dg4.reorder_levels(['ww','bz'])
print(data.date_dt)



dg5['yy'] = dg5['date_dt'].apply(get_year)

dg5ordered=dg5[col_order2]
dg5

AttributeError: 'SeriesGroupBy' object has no attribute 'get_year'

In [ ]:
dg5ordered#.sort_values('yy')


In [117]:
dg3

,ww,bz,vl,an,acw,yy,mm,tt,vb,ht
0,1,k,72,859,0.523171,2017,1,1.694167,787,2.217338
1,2,k,45,593,0.334572,2017,1,1.117350,548,1.451921
2,3,k,26,423,0.248611,2017,1,0.769525,397,1.018137
3,4,k,50,409,0.229954,2017,1,0.712419,359,0.942373
4,5,k,68,548,0.306169,2017,1,0.980208,480,1.286377
5,6,k,36,570,0.317882,2017,2,1.144769,534,1.462650
6,7,k,31,252,0.126586,2017,2,0.397072,221,0.523657
7,8,k,59,401,0.207326,2017,2,0.664525,342,0.871852
8,9,k,59,279,0.148715,2017,2,0.466944,220,0.615660
9,9,n,13,48,0.013194,2017,3,0.032905,35,0.046100


In [ ]:
data